# Setup

I'm using a separate version of python (maybe I shouldn't) and the only way I got this to work was to cd into the source directory.

In [1]:
cd /home/amemory/opt/Trio-1.1.3/trio/

/home/amemory/opt/Trio-1.1.3/trio


In [2]:
pwd

u'/home/amemory/opt/Trio-1.1.3/trio'

In [3]:
!ls

common.py  __init__.py	parser	sql	 trio.cfg   triodb.pyc	xtuple.py
executor   node.py	spi	test.py  triodb.py  visitor


In [36]:
import triodb
import pandas as pd

In [5]:
conn = triodb.connect(database='trio1', 
                      user='amemory', 
                      password='password')

In [6]:
#conn.close()

It seems that the DB always starts empty.

In [7]:
cur = conn.cursor()

# Single tuple

Create a single database, insert a single tuple, then query that tuple and see its lineage.

In [23]:
cur.execute("create table CRIME(type varchar(32), sector int)")

In [24]:
cur.execute("""insert into CRIME values ('Kidnapping', 1)""")

In [25]:
cur.execute("select * from crime")
tlst = cur.xfetchall()

In [27]:
tup = tlst[0]
tup

<trio.xtuple.XTuple instance at 0x7fafeac82830>

In [28]:
alt = tup.getAlternative(0)
alt

<trio.xtuple.Alternative instance at 0x7fafeac82bd8>

In [29]:
alt.getConfidence()

Not sure why there is no confidence and the computeConfidence call experiences an exception.

In [30]:
alt.computeConfidence(conn)

'error'

In [31]:
lin = alt.getLineage()
lin

<trio.xtuple.Lineage instance at 0x7fafeac82998>

In [32]:
lin.items()

[('crime', 1, 0)]

It seems that this lineage refers to id 1 of the crime relation.

In [33]:
lin.aids()

[1]

In [34]:
lin.aidstrs()

['1']

# Crime

In [76]:
conn.close()

In [9]:
conn = triodb.connect(database='trio1', 
                      user='amemory', 
                      password='password')
cur = conn.cursor()

In [40]:
# Causes an error -- the DB is empty again after closing and reopening
# cur.execute("select * from crime")
# tlst = cur.xfetchall()

## Load crime

In [10]:
sql = """
create table CRIME(type varchar(32), sector int);
insert into CRIME values ('Kidnapping', 1); 
insert into CRIME values ('Car theft', 2); 
insert into CRIME values ('Robbery', 2); 
create table WITNESS(name varchar(32), sector int);
insert into WITNESS values ('Mary', 2);
insert into WITNESS values ('Susan', 1);
insert into WITNESS values ('Jane', 1);
create table PERSON(name varchar(32), hair varchar(32), clothes varchar(32));
insert into PERSON values ('Jim', 'black', 'red');
insert into PERSON values ('Hank', 'brown', 'green');
insert into PERSON values ('Bob', 'gray', 'blue');
create trio table SAWPERSON(witness varchar(32), hair varchar(32), clothes varchar(32), uncertain(hair, clothes)); 
insert into SAWPERSON values ('Mary', 'black', ['red' | 'green']);
insert into SAWPERSON values ('Jane', 'gray', 'blue');
insert into SAWPERSON values ('Jane', 'brown', 'green') ?;
insert into SAWPERSON values [('Susan', 'gray', 'red') |('Susan', 'brown', 'red') |('Susan', 'brown', 'green')];
create trio table DRIVES(person varchar(32), car varchar(32), uncertain(person, car)) with confidences;
insert into DRIVES values [('Jim', 'Honda'):1];
insert into DRIVES values (['Hank':0.6 | 'Jim':0.3], 'Toyota':1);
insert into DRIVES values ('Bob':1, ['Toyota':0.5 | 'Mazda':0.3]);
create trio table SAWCAR(witness varchar(32), car varchar(32), uncertain(car)) with confidences;
insert into SAWCAR values [('Susan', 'Honda') :1];
insert into SAWCAR values ('Mary':1, 'Toyota':0.7);
insert into SAWCAR values [ ('Jane', 'Toyota' ) :0.5 | ('Jane', 'Honda') :0.4];
"""

In [11]:
for line in sql.split(";"):
    line = line.strip()
    line = line.replace("\n"," ")
    print line
    if line == "":
        continue
    cur.execute(line)

create table CRIME(type varchar(32), sector int)
insert into CRIME values ('Kidnapping', 1)
insert into CRIME values ('Car theft', 2)
insert into CRIME values ('Robbery', 2)
create table WITNESS(name varchar(32), sector int)
insert into WITNESS values ('Mary', 2)
insert into WITNESS values ('Susan', 1)
insert into WITNESS values ('Jane', 1)
create table PERSON(name varchar(32), hair varchar(32), clothes varchar(32))
insert into PERSON values ('Jim', 'black', 'red')
insert into PERSON values ('Hank', 'brown', 'green')
insert into PERSON values ('Bob', 'gray', 'blue')
create trio table SAWPERSON(witness varchar(32), hair varchar(32), clothes varchar(32), uncertain(hair, clothes))
insert into SAWPERSON values ('Mary', 'black', ['red' | 'green'])
insert into SAWPERSON values ('Jane', 'gray', 'blue')
insert into SAWPERSON values ('Jane', 'brown', 'green') ?
insert into SAWPERSON values [('Susan', 'gray', 'red') |('Susan', 'brown', 'red') |('Susan', 'brown', 'green')]
create trio table DRIVE

## Crime views

In [12]:
sql = """
create table SUSPECT_S as
select PERSON.name as suspect, CRIME.type as crime
from  SAWPERSON, WITNESS, CRIME, PERSON
where SAWPERSON.witness = WITNESS.name
  and CRIME.sector = WITNESS.sector 
  and PERSON.hair = SAWPERSON.hair
  and PERSON.clothes = SAWPERSON.clothes;

create table SUSPECT_D as 
select DRIVES.person as suspect, CRIME.type as crime
from  SAWCAR, WITNESS, CRIME, DRIVES
where SAWCAR.witness = WITNESS.name
  and WITNESS.sector = CRIME.sector
  and DRIVES.car = SAWCAR.car; 

create table HIGH_SUSPECT as
select SUSPECT_D.suspect, SUSPECT_D.crime
from SUSPECT_D, SUSPECT_S
where SUSPECT_D.suspect = SUSPECT_S.suspect 
  and SUSPECT_D.crime = SUSPECT_S.crime
  and conf(SUSPECT_D) >= 0.25;

create table ACCOMPLICES as 
select S1.suspect as name1, S2.suspect as name2 
from SUSPECT_D S1, SUSPECT_D S2
where S1.crime = S2.crime
  and S1.suspect < S2.suspect;
"""

In [13]:
for line in sql.split(";"):
    line = line.strip()
    line = line.replace("\n"," ")
    print line
    if line == "":
        continue
    cur.execute(line)

create table SUSPECT_S as select PERSON.name as suspect, CRIME.type as crime from  SAWPERSON, WITNESS, CRIME, PERSON where SAWPERSON.witness = WITNESS.name   and CRIME.sector = WITNESS.sector    and PERSON.hair = SAWPERSON.hair   and PERSON.clothes = SAWPERSON.clothes
SELECT: 0.00501799583435
INDEX:  0
create table SUSPECT_D as  select DRIVES.person as suspect, CRIME.type as crime from  SAWCAR, WITNESS, CRIME, DRIVES where SAWCAR.witness = WITNESS.name   and WITNESS.sector = CRIME.sector   and DRIVES.car = SAWCAR.car
SELECT: 0.00858497619629
INDEX:  0
create table HIGH_SUSPECT as select SUSPECT_D.suspect, SUSPECT_D.crime from SUSPECT_D, SUSPECT_S where SUSPECT_D.suspect = SUSPECT_S.suspect    and SUSPECT_D.crime = SUSPECT_S.crime   and conf(SUSPECT_D) >= 0.25
SELECT: 0.00472402572632
INDEX:  0
create table ACCOMPLICES as  select S1.suspect as name1, S2.suspect as name2  from SUSPECT_D S1, SUSPECT_D S2 where S1.crime = S2.crime   and S1.suspect < S2.suspect
SELECT: 0.00794291496277
INDE

## Functions

In [13]:
def grabtupleBadWay(tablename,aid,cur):
        cur.execute("""SELECT * from %s""" % (tablename,))
        lst = cur.xfetchall()
        tup = lst[aid-1]
        return tup
        
def disjToString(disj):
        if disj==0:
                return "and"
        elif disj==1:
                return "or"
        elif disj==2:
                return "and not"
        else:
                return "error"

def buildLinageBadWay(linage,cur,conn):
        explanation = "";
        formula = []
        i = 0
        for elem in linage.items():
                (tablename,aid,disj) = elem
                tup = grabtupleBadWay(tablename,aid,cur)
                if len(tup)>0:
                        alt = tup.getAlternative(0)
                        if alt.getQuestionMark():
                                conf = alt.computeConfidence(conn)
                        else:
                                conf = float(1)
                        newelem = (conf, disj)
                        formula.append(newelem)
                        srtup = str(tup[0])+": "+str(conf)
                        explanation = explanation + srtup +" "+disjToString(disj)+" "
                else:
                        raise Exception("No tuple: "+str(tablename)+" "+str(aid)+" exists!")
        print explanation
        return formula


In [14]:
def q(sql):
    return sql.replace("\n"," ").replace(";","")

## Tables

In [68]:
cur.execute(q("""select * from crime"""))
for t in cur.xfetchall(): print t

(Kidnapping,1)

(Car theft,2)

(Robbery,2)



In [69]:
cur.execute(q("""select * from witness"""))
for t in cur.xfetchall(): print t

(Mary,2)

(Susan,1)

(Jane,1)



In [70]:
cur.execute(q("""select * from person"""))
for t in cur.xfetchall(): print t

(Jim,black,red)

(Hank,brown,green)

(Bob,gray,blue)



In [71]:
cur.execute(q("""select * from sawperson"""))
for t in cur.xfetchall(): print t

(Mary,black,green)|(Mary,black,red)

(Jane,gray,blue)

(Jane,brown,green)?

(Susan,brown,green)|(Susan,brown,red)|(Susan,gray,red)



In [72]:
cur.execute(q("""select * from drives"""))
for t in cur.xfetchall(): print t

(Jim,Honda)

(Hank,Toyota)?|(Jim,Toyota)?

(Bob,Mazda)?|(Bob,Toyota)?



In [73]:
cur.execute(q("""select * from sawcar"""))
for t in cur.xfetchall(): print t

(Susan,Honda)

(Mary,Toyota)?

(Jane,Honda)?|(Jane,Toyota)?



In [74]:
cur.execute(q("""select * from SUSPECT_S"""))
for t in cur.xfetchall(): print t

(Jim,Car theft)?

(Jim,Robbery)?

(Bob,Kidnapping)

(Hank,Kidnapping)?

(Hank,Kidnapping)?



In [75]:
cur.execute(q("""select * from SUSPECT_D"""))
for t in cur.xfetchall(): print t

(Jim,Kidnapping)

(Hank,Car theft)?|(Jim,Car theft)?

(Bob,Car theft)?

(Hank,Robbery)?|(Jim,Robbery)?

(Bob,Robbery)?

(Jim,Kidnapping)?

(Hank,Kidnapping)?|(Jim,Kidnapping)?

(Bob,Kidnapping)?



## Query

In [57]:
sql = """
select suspect, crime
from HIGH_SUSPECT H, WITNESS W
where H ==> W
and W.name = 'Susan';
"""

In [58]:
cur.execute(q(sql))
tlst = cur.xfetchall()

In [59]:
def q(sql):
    return sql.replace("\n"," ").replace(";","")

In [60]:
tup = tlst[0]
tup

<trio.xtuple.XTuple instance at 0x7ff12e68f8c0>

In [61]:
print tup

(Hank,Kidnapping)?



In [62]:
alt = tup.alternatives[0]
alt

<trio.xtuple.Alternative instance at 0x7ff12e68f200>

In [63]:
lin = alt.getLineage()
lin

<trio.xtuple.Lineage instance at 0x7ff12e68f488>

In [64]:
lin.items()

[('high_suspect', 2, 0), ('witness', 2, 0)]

Not sure this lineage makes sense yet, but it seems to be a conjunction of the 2nd tuple of high_suspect and the 2nd tuple of witness.

In [65]:
print alt

(Hank,Kidnapping)?


In [66]:
buildLinageBadWay(lin, cur, conn)

(Hank,Kidnapping)?: 0.1 and (Susan,1): 1.0 and 


[(0.1, 0), (1.0, 0)]

# Tuple-uncertain crime

In [8]:
conn.close()

In [9]:
conn = triodb.connect(database='trio1', 
                      user='amemory', 
                      password='password')
cur = conn.cursor()

In [10]:
sql = """
create table CRIME(type varchar(32), sector int);
insert into CRIME values ('Kidnapping', 1); 
insert into CRIME values ('Car theft', 2); 
insert into CRIME values ('Robbery', 2); 
create table WITNESS(name varchar(32), sector int);
insert into WITNESS values ('Mary', 2);
insert into WITNESS values ('Susan', 1);
insert into WITNESS values ('Jane', 1);
create table PERSON(name varchar(32), hair varchar(32), clothes varchar(32));
insert into PERSON values ('Jim', 'black', 'red');
insert into PERSON values ('Hank', 'brown', 'green');
insert into PERSON values ('Bob', 'gray', 'blue');
create trio table SAWPERSON(witness varchar(32), hair varchar(32), clothes varchar(32), uncertain(hair, clothes)); 
insert into SAWPERSON values ('Mary', 'black', 'red') ?;
insert into SAWPERSON values ('Mary', 'black', 'green') ?;
insert into SAWPERSON values ('Jane', 'gray', 'blue');
insert into SAWPERSON values ('Jane', 'brown', 'green') ?;
insert into SAWPERSON values ('Susan', 'gray', 'red') ?;
insert into SAWPERSON values ('Susan', 'brown', 'red') ?;
insert into SAWPERSON values ('Susan', 'brown', 'green') ?;
create trio table DRIVES(person varchar(32), car varchar(32), uncertain(person, car)) with confidences;
insert into DRIVES values [('Jim', 'Honda'):1];
insert into DRIVES values [('Hank', 'Toyota'):0.6];
insert into DRIVES values [('Jim', 'Toyota'):0.3];
insert into DRIVES values [('Bob','Toyota'):0.5];
insert into DRIVES values [('Bob','Mazda'):0.3];
create trio table SAWCAR(witness varchar(32), car varchar(32), uncertain(car)) with confidences;
insert into SAWCAR values [('Susan', 'Honda'):1];
insert into SAWCAR values [('Mary', 'Toyota'):0.7];
insert into SAWCAR values [('Jane', 'Toyota'):0.5];
insert into SAWCAR values [('Jane', 'Honda'):0.4];
"""

In [43]:
drives = pd.DataFrame([['Jim','Honda',1.0],
                       ['Hank','Toyota',.6],
                       ['Jim','Toyota',.3]],columns=['person','car','conf'])

In [44]:
drives

,person,car,conf
0,Jim,Honda,1.0
1,Hank,Toyota,0.6
2,Jim,Toyota,0.3


In [10]:
sql = """
create trio table DRIVES(person varchar(32), car varchar(32), uncertain(person, car)) with confidences;
insert into DRIVES values [('Jim', 'Honda'):1];
insert into DRIVES values [('Hank', 'Toyota'):0.6];
insert into DRIVES values [('Jim', 'Toyota'):0.3];
insert into DRIVES values [('Bob','Toyota'):0.5];
insert into DRIVES values [('Bob','Mazda'):0.3];
create trio table SAWCAR(witness varchar(32), car varchar(32), uncertain(car)) with confidences;
insert into SAWCAR values [('Susan', 'Honda'):1];
insert into SAWCAR values [('Mary', 'Toyota'):0.7];
insert into SAWCAR values [('Jane', 'Toyota'):0.5];
insert into SAWCAR values [('Jane', 'Honda'):0.4];
"""

In [11]:
for line in sql.split(";"):
    line = line.strip()
    line = line.replace("\n"," ")
    print line
    if line == "":
        continue
    cur.execute(line)

create trio table DRIVES(person varchar(32), car varchar(32), uncertain(person, car)) with confidences
insert into DRIVES values [('Jim', 'Honda'):1]
insert into DRIVES values [('Hank', 'Toyota'):0.6]
insert into DRIVES values [('Jim', 'Toyota'):0.3]
insert into DRIVES values [('Bob','Toyota'):0.5]
insert into DRIVES values [('Bob','Mazda'):0.3]
create trio table SAWCAR(witness varchar(32), car varchar(32), uncertain(car)) with confidences
insert into SAWCAR values [('Susan', 'Honda'):1]
insert into SAWCAR values [('Mary', 'Toyota'):0.7]
insert into SAWCAR values [('Jane', 'Toyota'):0.5]
insert into SAWCAR values [('Jane', 'Honda'):0.4]



In [16]:
cur.execute(q("""
create table seencars as
select drives.person as suspect, drives.car, witness from drives, sawcar
where drives.car = sawcar.car;
"""))
for t in cur.xfetchall(): print t

SELECT: 0.00598502159119
INDEX:  0


In [35]:
cur.execute(q("""select * from seencars"""))
for t in cur.xfetchall(): 
    print str(t).strip(), t.alternatives[0].computeConfidence(conn)

(Jim,Honda,Susan) 1.0
(Jim,Honda,Jane)? 0.4
(Hank,Toyota,Mary)? 0.42
(Hank,Toyota,Jane)? 0.3
(Jim,Toyota,Mary)? 0.21
(Jim,Toyota,Jane)? 0.15
(Bob,Toyota,Mary)? 0.35
(Bob,Toyota,Jane)? 0.25


In [42]:
rows = []
cur.execute(q("""select * from seencars"""))
for t in cur.xfetchall(): 
    #print str(t).strip(), t.alternatives[0].computeConfidence(conn)
    rows.append({'tuple':str(t).strip(), 'conf':t.alternatives[0].computeConfidence(conn)})
df = pd.DataFrame(rows)
df.sort('conf', ascending=False)

,conf,tuple
0,1.00,"(Jim,Honda,Susan)"
2,0.42,"(Hank,Toyota,Mary)?"
1,0.40,"(Jim,Honda,Jane)?"
6,0.35,"(Bob,Toyota,Mary)?"
3,0.30,"(Hank,Toyota,Jane)?"
7,0.25,"(Bob,Toyota,Jane)?"
4,0.21,"(Jim,Toyota,Mary)?"
5,0.15,"(Jim,Toyota,Jane)?"
